<a href="https://colab.research.google.com/github/devansong/network_workshop/blob/master/Wildlife_Trade_Network_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# URI Illegal Wildlife Trafficking: R workshop in network analysis

## Setup

This section installs and loads all the packages we need, and downloads the CITES dataset using the [citesdb](https://github.com/ropensci/citesdb) package.  We'll call this dataset `master`.

Click on the cell and press the play button, or press shift+enter to run the code

This takes approximately 10-15 minutes to run. We will return to the powerpoint introductions after you run this code.

In [0]:
install.packages("igraph")
devtools::install_github("ropensci/citesdb")

library(igraph)
library(tidyverse)
library(citesdb)

# Collect dataset, and remove NA's from the Importer/Exporter columns
cites_db_download()
master <- cites_shipments() %>% 
    collect()
master$Importer[is.na(master$Importer)] <- ""
master$Exporter[is.na(master$Exporter)] <- ""

## Activity 0: Test
Before we begin the class, let's make sure we can run some simple code and plot a graph. 
Click on each cell of code and press "shift + enter"

In [0]:
# Anything in a coding cell that has a "#" in front is just a commeent about the code

# I am creating a variable x, that consists of 5 different numbers
x <- c(1,3,6,9,12)

# I am now creating a variable y, that consists of 5 different numbers
y <- c(1.5,2,7,8,15)

#I now want to plot x against y in a simple scatter plot 
plot(
    x, y, 
    xlab="x axis", 
    ylab="y axis", 
    main="My nice little plot with fairly large blue dots",
    ylim=c(0,20),
    xlim=c(0,15), 
    pch=19,
    col="blue",
    cex=3
)

If you see a simple scatter plot with 5 data points, you have successfully run R code!

##Activity 1: Examine CITES data

### 1.1 Explore the structure (str) of the CITES dataset

In [0]:
str(master)

### 1.2 Print the first 6 rows of data

In [0]:
head(master)

### 1.3 Explore other properties of the dataset

In [0]:
min(master$Year) 

In [0]:
max(master$Year) 

In [0]:
table(master$Class) 

In [0]:
sort(table(master$Class))

In [0]:
plot(sort(table(master$Class)))

Answer the following questions: 

a. how many shipments does your data have?  
b. how many different classes are represented in your dataset?  
c. what timeframe does your dataset span? 


## Activity 2: Convert CITES data to network and visualize

##### 2.1 Run Functions

These functions will be used throughout this workshop to make things easier.

`create_network` makes an igraph network from a dataset, which we can use for further analysis and visualization.

`plot_network` allows us to plot an igraph network in a nicer way than by default, and easily change the edge weights

In [0]:
create_network <- function(data){
    # create a tibble of unique Exporter and Importer countries
    # this is used for the network nodes
    sources <- data %>%
        distinct(Exporter) %>%
        rename(label = Exporter)  # all exporting countries (treated as country of origin)
    destinations <- data %>%
        distinct(Importer) %>%
        rename(label = Importer)  # all importing countries
    nodes <- full_join(sources, destinations, by = "label")  # all unique countries in your dataset 
    nodes <- nodes %>%
        rowid_to_column("id")  # create numerical id for each unique country 

    # create a tibble with export id, import id, and weight by number of shipments
    # this is used for the network edges
    per_route <- data %>%  
        group_by(Exporter, Importer) %>%
        summarise(weight = n()) %>% #number of observations in the current group
        ungroup() #removes grouping 
    edges <- per_route %>% 
        left_join(nodes, by = c("Exporter" = "label")) %>% 
        rename(from = id)
    edges <- edges %>% 
        left_join(nodes, by = c("Importer" = "label")) %>% 
        rename(to = id)
    edges <- select(edges, from, to, weight)

    # create a directed igraph network from the edges and nodes defined above   
    net <- graph_from_data_frame(d=edges, vertices=nodes, directed=T)
    return(net)
}


plot_network <- function(
    igraph_network,
    title="trade network",
    weight_factor=0.01, # scaling factor for edge weights
    layout=layout_with_mds  # igraph layour
    ){
    E(igraph_network)$width <- E(igraph_network)$weight*weight_factor
    l <- layout(igraph_network)
    plot(
        # TODO: adjust plot size, etc to make look niceer in Colab
        igraph_network,
        edge.color= rgb(70/255, 130/255, 180/255, 0.6),
        edge.arrow.size=.001, 
        vertex.size = 13, 
        vertex.color= rgb(255/255,190/255,0/255, 0.8),
        vertex.label = V(igraph_network)$id,
        vertex.label.cex = 0.6, 
        vertex.label.color = "black", 
        rescale=T, 
        layout=l*3.0, 
        main=title) 
}

### 2.2 Create and visualize network

Now we'll start to explore the network, where each node is a country. See Annex 3 of the [CITES Trade Database Guide](https://trade.cites.org/cites_trade_guidelines/en-CITES_Trade_Database_Guide.pdf) to look up the country name from its 2-letter code.

In [0]:
master_network <-create_network(master) 

In [0]:
plot_network(master_network, weight_factor=0.00002, layout = layout_randomly)

What an uninformative mess! This doesn't really tell us anything. 
We'll remove some nodes (such as Unknown or blank country names), to clean up our dataset.

In [0]:
# get vertex ID of ambiguous country codes
unknown <- which(V(master_network)$label == "XX")
various <- which(V(master_network)$label == "XV")
blank <- which(V(master_network)$label == "")
# delete them from our network
master_network <- delete_vertices(master_network, c(unknown, blank, various))

Next, we'll remove nodes with weights less than 2000, so we can see the major nodes in our network.

In [0]:
s1 <- subgraph.edges(master_network, E(master_network)[E(master_network)$weight>2000], del=F)
s2 <- delete_vertices(s1, degree(s1, mode = "in")==0)
plot_network(s2, weight_factor=0.00002, layout = layout_randomly)

Much better! You can right click on the figure above and open in a new tab to zoom in

There are many different network layouts in igraph. Let's try some of them:

In [0]:
plot_network(s2, weight_factor=0.00002, layout = layout_on_sphere)

In [0]:
plot_network(s2, weight_factor=0.00002, layout = layout_with_mds)

In [0]:
plot_network(s2, weight_factor=0.00002, layout = layout.circle)


Which layouts do you like? And why? Is there anything else about this graph that you would like to see? 

Type in your answer here: 

## Activity 3: Compare subsets of Amphibian data

### 3.1 Subset 2 datasets, convert to network, and visualize

In [0]:
Amp_1992 <- subset(master, Class == "Amphibia" & Year == "1992")  # Create subset of Amphibia data from 1992
Amp_2012 <- subset(master, Class == "Amphibia" & Year == "2012")  # Create subset of Amphibia data from 2012

Question: Do you think that Amphibian trade in 1992 will be different than Amphibian trade in 2012? What are some suggested differences?



Write your answer here

In [0]:
# Create networks from these subsets
net1 <-create_network(Amp_1992)
net2 <-create_network(Amp_2012)

In [0]:
par(mfrow=c(1,2), mar = c(0.1, 0.1, 1, 0.1))  # create a graph with 1 row and 2 columns of plots
plot_network(net1, title="1992", weight_factor=0.2, layout= layout_on_sphere)
plot_network(net2, title="2012", weight_factor=0.2, layout= layout_on_sphere)

### 3.2 Visual cluster analysis of subsets

In [0]:
par(mfrow=c(1,2), mar = c(0.1, 0.1, 1, 0.1))  # create a graph with 1 row and 2 columns of plots
cfg <- cluster_fast_greedy(as.undirected(net1))  # run clustering algorthim on our first network
plot(cfg, as.undirected(net1)) 
title(main = "Amphibians 1992")
cfg <- cluster_fast_greedy(as.undirected(net2)) # run clustering algorthim on our first network
plot(cfg, as.undirected(net2)) 
title(main = "Amphibians 2012")

Compare the clusters from 1992 and 2012. Each color represents a module, or a clique of countries that are similar to each other. How are the cliques different between 1992 and 2012? What do you think is happening to the amphibian trade over time? 

Write your answer here:

## Activity 4: Network and node descriptive stats

Visually inspecting your network can be fun, but it doesn't give you a quantitative description of your data. For example, if you have a straight trendline through an x-y scattr plot, it is still useful to find out numerical properties like the mean, variance, slope of regression line, intercept, etc. 

For the next few examples, we are only going to examine the properties of net1, which is the amphibian trade in 1992

**Density:** The proportion of present edges from all possible ties.

In [0]:
ecount(net1)/(vcount(net1)*(vcount(net1)-1)) #for a directed network

**Reciprocity:** The proportion of reciprocated ties (for a directed network).

In [0]:
reciprocity(net1)

**Global Transitivity:** ratio of triangles (direction disregarded) to connected triples

In [0]:
transitivity(net1, type="global")

**Diameter:** longest geodesic distance
Note that edge weights are used by default, unless set to NA.

In [0]:
diameter(net1, directed=F, weights=NA)

**Node degree:** Number of connections each node has to other nodes'

In [0]:
deg <- degree(net1, mode="all") #NODE METRICS ************
plot(net1, vertex.size=deg*3, edge.arrow.size=0.1)
hist(deg, breaks=1:vcount(net1)-1, main="Histogram of node degree")

**Hubs:** lots of outgoing links


In [0]:
hs <- hub_score(net1, weights=NA)$vector
plot(net1, vertex.size=hs*50, main="Hubs", edge.arrow.size=0.1, vertex.color= rgb(212/255, 175/255, 55/255, 0.6))


**Authorities:** lots of incoming links from hubs

In [0]:
as <- authority_score(net1, weights=NA)$vector
plot(net1, vertex.size=as*30, main="Authorities", edge.arrow.size=0.1, vertex.color= rgb(212/255, 175/255, 55/255, 0.6)) 

**Community detection**

In [0]:
clp <- cluster_label_prop(net1)
plot(clp, net1, edge.arrow.size=0.1)

In [0]:
V(net1)$community <- cfg$membership
colrs <- adjustcolor( c("gray50", "tomato", "gold", "yellowgreen", "blue", "gold", "maroon"), alpha=.6)
plot(net1, vertex.color=colrs[V(net1)$community])

## Activity 5: If we have time, compare network metrics with two different subsets of your choice

*You can pick any two logical subsets to compare, or choose one of the following suggested pairs. Make sure you actually have data by examining the structure of your subsets! hint:
table(yoursubset$Year) to see how many entries it has*

Don't use entries later than 2016 as data is incomplete


Option 1: Insects in 1988 vs Insects in 2016

Option 2: African elephants vs Indian elephants

Option 3: African elephants 1990 vs 2015

Option 4: Tortoises in the 1990s vs Tortoises in the 2000s 



###For those of you who want to code by yourself, ignore my code below and copy and paste Activity 3's code and modify

In [0]:
##Insert your own code here, if you want to try coding yourself

If you prefer to run my code, let's look at 4 examples below! 

In [0]:
###############################################
#Option 1: Insects in 1988 vs Insects in 2016
###############################################
Insect1988 <- subset(master, Class == "Insecta" & Year == "1988")  # Create subset of Insect data from 1988
Insect2016 <- subset(master, Class == "Insecta" & Year == "2016")  # Create subset of Insect data from 2016
table(Insect1988$Year)
table(Insect2016$Year)
Ins1 <-create_network(Insect_1988) #Create network from 1988 insect data 
Ins2 <-create_network(Insect_2016) #Create network from 2016 insect data 
par(mfrow=c(1,2), mar = c(0.1, 0.1, 1, 0.1))  # create a graph with 1 row and 2 columns of plots
plot_network(Ins1, title="Insects 1988", weight_factor=0.1, layout= layout_on_sphere) #plot graph 
plot_network(Ins2, title="Insects 2016", weight_factor=0.1, layout= layout_on_sphere) # plot graph

In [0]:
###############################################
#Option 2: African elephants vs Indian elephants
###############################################

African <- subset(master, Genus == "Loxodonta")
Indian <- subset (master, Genus== "Elephas")
table(African$Year)
table(Indian$Year)
net1 <-create_network(African) 
net2 <-create_network(Indian) 
s1 <- subgraph.edges(net1, E(net1)[E(net1)$weight>10], del=F)
net1 <- delete_vertices(s1, degree(s1, mode = "in")==0)
s2 <- subgraph.edges(net2, E(net2)[E(net2)$weight>10], del=F)
net2 <- delete_vertices(s2, degree(s2, mode = "in")==0)
par(mfrow=c(1,2), mar = c(0.1, 0.1, 1, 0.1))  # create a graph with 1 row and 2 columns of plots
plot_network(net1, title="African Elephants", weight_factor=0.001, layout= layout_on_sphere) #plot graph 
plot_network(net2, title="Indian Elepahtns", weight_factor=0.001, layout= layout_on_sphere) # plot graph

In [0]:
###############################################
#Option 3: African elephants 1990 vs 2015
###############################################
Lox1990 <- subset(master, Genus == "Loxodonta" & Year ==1990)
Lox2015 <- subset(master, Genus == "Loxodonta" & Year ==2015)
table(Lox1990$Year)
table(Lox2015$Year)
net1 <-create_network(Lox1990) 
net2 <-create_network(Lox2015) 
s1 <- subgraph.edges(net1, E(net1)[E(net1)$weight>5], del=F)
net1 <- delete_vertices(s1, degree(s1, mode = "in")==0)
s2 <- subgraph.edges(net2, E(net2)[E(net2)$weight>5], del=F)
net2 <- delete_vertices(s2, degree(s2, mode = "in")==0)
par(mfrow=c(1,2), mar = c(0.1, 0.1, 1, 0.1))  
plot_network(net1, title="African Elephants 1990", weight_factor=0.06, layout= layout_on_sphere) #plot graph 
plot_network(net2, title="African Elephants 2015", weight_factor=0.06, layout= layout_on_sphere) # plot graph

In [0]:
###############################################
#Option 4: Tortoises in the 1990s vs Tortoises in the 2000s 
###############################################
Torts90s <- subset(master, Family == "Testudinidae" & Year > 1989 & Year < 2000 )
Torts20s <- subset(master, Family == "Testudinidae" & Year > 1999 & Year < 2010 )
table(Torts90s$Year)
table(Torts20s$Year)
net1 <-create_network(Torts90s)
net2 <-create_network(Torts20s)
s1 <- subgraph.edges(net1, E(net1)[E(net1)$weight>10], del=F)
net1 <- delete_vertices(s1, degree(s1, mode = "in")==0)
s2 <- subgraph.edges(net2, E(net2)[E(net2)$weight>10], del=F)
net2 <- delete_vertices(s2, degree(s2, mode = "in")==0)
par(mfrow=c(1,2), mar = c(0.1, 0.1, 1, 0.1))  # create a graph with 1 row and 2 columns of plots
plot_network(net1, title="Tortoises 90s", weight_factor=0.02, layout= layout_on_sphere)
plot_network(net2, title="Tortoises 2000s", weight_factor=0.02, layout= layout_on_sphere)

## References and Further Reading
Csardi G, Nepusz T: The igraph software package for complex network research, InterJournal, Complex Systems 1695. 2006.
  http://igraph.sf.net

Kolaczyk, E.D. and Csárdi, G., 2014. Statistical analysis of network data with R (Vol. 65). New York, NY: Springer.

Luke, D.A., 2015. A user's guide to network analysis in R. Cham, Switzerland: Springer.

Ognyanova, K. (2016) Network analysis with R and igraph: NetSci X Tutorial. Retrieved from www.kateto.net/networks-r-igraph.

Ognyanova, K. (2019) Network visualization with R. Retrieved from www.kateto.net/network-visualization.

Ross, Noam, Evan A. Eskew, and Nicolas Ray. 2019. citesdb: A high-performance database of shipment-level CITES trade data. R package v0.2.0. EcoHealth Alliance: New York, NY. https://github.com/ropensci/citesdb. doi:10.5281/zenodo.2630836

UNEP-WCMC (Comps.) 2019. Full CITES Trade Database Download. Version 2019.2. CITES Secretariat, Geneva, Switzerland. Compiled by UNEP-WCMC, Cambridge, UK. Available at: https://trade.cites.org.

